In [0]:
#import tensorflow as tf 
#tf.test.gpu_device_name() 

In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [4]:
from keras import models
from keras import layers
from keras import regularizers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [5]:
from google.colab import drive 
drive.mount('/mntDrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /mntDrive


In [6]:
from keras.applications.densenet import DenseNet121
ds= DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in ds.layers[:-4]:
   layer.trainable = False
for layer in ds.layers:
    print(layer, layer.trainable)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


29089792/29084464 [==============================] - 1s 0us/step
<keras.engine.input_layer.InputLayer object at 0x7f2306286be0> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f23419be518> False
<keras.layers.convolutional.Conv2D object at 0x7f23419be438> False
<keras.layers.normalization.BatchNormalization object at 0x7f2306286f98> False
<keras.layers.core.Activation object at 0x7f2306286fd0> False
<keras.layers.convolutional.ZeroPadding2D object at 0x7f2306246e48> False
<keras.layers.pooling.MaxPooling2D object at 0x7f22f04adac8> False
<keras.layers.normalization.BatchNormalization object at 0x7f22f04d9eb8> False
<keras.layers.core.Activation object at 0x7f2306286e10> False
<keras.layers.convolutional.Conv2D object at 0x7f22f0465358> False
<keras.layers.normalization.BatchNormalization object at 0x7f22f047fb00> False
<keras.layers.core.Activation object at 0x7f22f0486828> False
<keras.laye

In [9]:
model = models.Sequential()
model.add(ds)

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

train_datagen = ImageDataGenerator(
                                   rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = train_datagen.flow_from_directory("/mntDrive/My Drive/COVID19/data/train",
                                                 target_size=(224, 224),
                                                 batch_size=16,
                                                 class_mode='binary')
test_dir = test_datagen.flow_from_directory("/mntDrive/My Drive/COVID19/data/test",
                                            target_size=(224, 224),
                                            batch_size=16,
                                            class_mode='binary',shuffle=False)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

history = model.fit_generator(
                              train_dir,
                              steps_per_epoch=245 ,
                              epochs=10,
                              validation_data=test_dir,
                              validation_steps=70,
                              verbose=1)



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1025      
Total params: 7,038,529
Trainable params: 39,937
Non-trainable params: 6,998,592
_________________________________________________________________
Found 245 images belonging to 2 classes.
Found 70 images belonging to 2 classes.
Epoch 1/10
245/245 [==============================] - 58s 236ms/step - loss: 0.2814 - acc: 0.8774 - val_loss: 2.6190 - val_acc: 0.7000
Epoch 2/10
245/245 [==============================] - 53s 216ms/step - loss: 0.1849 - acc: 0.9281 - val_loss: 2.8087 - val_acc: 0.7143
Epoch 3/10
245/245

In [12]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
# Below here is the code snippet for calculating various performance parameters of our model().
tpr=[]
fpr=[]
total_no_fp=0
total_no_tp=0
total_no_fn=0
total_no_tn=0
total=[]
precision=[]
f1=[]
acc=[]
sp=[]
validation_path1="/mntDrive/My Drive/COVID19/data/validateAll/val"
for i in range(1):
	valid_datagen = ImageDataGenerator(rescale=1./255)
	valid_dir = valid_datagen.flow_from_directory(validation_path1+str(i),
		                                 target_size=(224, 224),
			                            batch_size=16,
			                            class_mode='binary',shuffle=False)
	Y_pred = model.predict_generator(valid_dir, steps= len(valid_dir))
	print("\n Y_pred ","\n", Y_pred)
	XtrueX = valid_dir.classes
	y_pred = []
	for it1 in range(len(Y_pred)):
		if Y_pred[it1]>0.5:
			y_pred.append(1)
			if XtrueX[it1]==1:
				total_no_tp = total_no_tp + 1
			else:
				total_no_fp = total_no_fp + 1
		else:
			y_pred.append(0)
			if XtrueX[it1] == 1:
				total_no_fn = total_no_fn + 1
			else:
				total_no_tn = total_no_tn + 1
	print("case == ", i)
	print("valid_dir.classes=",valid_dir.classes)
	print("     y_pred      =",y_pred)
	Conf_Matrix = confusion_matrix(valid_dir.classes, y_pred)#.ravel()
	tpr.append( Conf_Matrix[1][1] /(Conf_Matrix[1][1]+Conf_Matrix[1][0]) ) #tpr=tp/(tp+fn)
	fpr.append( Conf_Matrix[0][1] /(Conf_Matrix[0][0]+Conf_Matrix[0][1]) ) #fpr=fp/(tn+fp)
	current=len(fpr)-1

	for it in range(len(fpr)):
		if len(fpr)==1:
			break
		if fpr[len(fpr)-it-2] <= fpr[current]:
			break
		else:
			temp=fpr[current]
			fpr[current]=fpr[len(fpr)-it-2]
			fpr[len(fpr)-it-2]=fpr[current]

			temp=tpr[current]
			tpr[current]=tpr[len(tpr)-it-2]
			tpr[len(tpr)-it-2]=tpr[current]

			current=len(tpr)-it-2

	pr=  Conf_Matrix[1][1] /(Conf_Matrix[1][1]+Conf_Matrix[0][1]) 
  #pr=tp/(tp+fp)
	precision.append(pr)
	f1.append( (2*Conf_Matrix[1][1]) / (2*Conf_Matrix[1][1] + Conf_Matrix[0][1] + Conf_Matrix[1][0])  )
	#f1 = 2*tp / (2*tp+fp+fn)
	acc.append( (Conf_Matrix[1][1]+Conf_Matrix[0][0])/(Conf_Matrix[0][0]+Conf_Matrix[0][1]+Conf_Matrix[1][0]+Conf_Matrix[1][1]) )
	#acc = (tp+tn) / (tp+tn+fp+fn)
	sp.append( Conf_Matrix[0][0] /(Conf_Matrix[0][0]+Conf_Matrix[0][1]) )
	#sp = tn / (tn+fp)
print("total false positive = ",total_no_fp,
      "\ntotal  true positive = ",total_no_tp,
			"\ntotal false negative = ",total_no_fn,
			"\ntotal  true negative = ",total_no_tn)
print("All false positive rates and true positive rates")
for itn in range(1):
	print("fpr=",fpr[itn],"; tpr=",tpr[itn], "; acc=", acc[itn], "; Recall=", tpr[itn], "; Precesion=", precision[itn], "; F1=", f1[itn], "; Specificity=", sp[itn])
print("Mean Recall is", sum(tpr)/len(tpr))
print("Mean Accuracy is", sum(acc)/len(acc))
print("Precision Macro is", sum(precision)/len(precision))
print("Mean F1 score is", sum(f1)/len(f1))
print("Mean Specificity is", sum(sp)/len(sp))

#plt.figure(1)
#plt.plot([0, 1], [0, 1], 'k--')

#from sklearn.metrics import auc
#auc_keras = auc(fpr, tpr)


#plt.plot(fpr, tpr, label='Keras (area = {:.3f})'.format(auc_keras))

#plt.xlabel('False positive rate')
#plt.ylabel('True positive rate')
#plt.title('ROC curve')
#plt.legend(loc='best')
#plt.show()

Found 36 images belonging to 2 classes.

 Y_pred  
 [[3.0785799e-05]
 [5.4180622e-05]
 [5.7816505e-06]
 [4.6253204e-05]
 [4.6551228e-05]
 [3.4451485e-05]
 [2.4259090e-05]
 [2.5719404e-05]
 [7.8082085e-06]
 [2.6971102e-05]
 [4.3720007e-05]
 [8.5532665e-06]
 [4.9382448e-05]
 [5.7548285e-05]
 [2.4080276e-05]
 [1.2099743e-05]
 [4.5895576e-06]
 [2.0503998e-05]
 [1.8399954e-04]
 [3.1322241e-05]
 [3.5434961e-05]
 [3.1769276e-05]
 [9.2625618e-05]
 [4.0084124e-05]
 [4.4137239e-05]
 [3.4958124e-05]
 [1.5810132e-04]
 [4.7910810e-03]
 [5.8749318e-04]
 [1.4385581e-04]
 [9.0122223e-05]
 [3.0756593e-03]
 [3.3250451e-04]
 [3.9088726e-04]
 [3.3350229e-02]
 [1.7780364e-03]]
case ==  0
valid_dir.classes= [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1]
     y_pred      = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
total false positive =  0 
total  true positive =  0 
total false negative =  12 
total  true negative 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in long_scalars
